In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import numba

Working with Binary Images

In [ ]:
image = Image.open("")
data = np.array(image).astype(np.float32) / 255.

In [ ]:
path = ""
image = Image.open(path)
data = np.array(image).astype(np.float32) / 255.
if len(data.shape) == 2:
    data = data[:, :, np.newaxis]

In [ ]:
plt.show(data[:,:,0], cmap='Greys_r')

Traditional approach: rounding the images between 0 and 1. Increases contrast of the image and results in bad outcome. Almost can't recognise actual image.

In [ ]:
data_rounded = np.round(data)
plt.imshow(data_rounded[:, :, 0], cmap='Greys_r')

We are going to apply ditering algorithm to fix traditional approach by rounding surrounding pixels and reducing average error rate. Making a perceiveness of actual image (original image).

In [ ]:
@numba.jit("f4[:, :, :]f4[:, :, :]", nopython=True, nogil=True)
def floyd_steinberg(image):
    Lx, Ly, Lc = image.shape
    for j in range(Ly):
        for i in range(Lx):
            for c in range(Lc):
                rounded = round(image[i, j, c])
                err = image[i, j, c] - rounded
                image[i, j, c] = rounded
                if i < Lx-1: image[i+1, j, c] += (7/16)*err
                if j<Ly-1L
                    image[i, j+1, c] += (5/16)*err
                    if i > 0: image[i-1, j+1, c] += (1/16)*err
                    if i<Lx-1: image[i+1, j+1, c] += (3/16)*err
    return image



In [ ]:
data_fs = floyd_steinberg(data.copy())
plt.imshow(data_fs[:,:,0], cmap='Greys_r')

Now, we will write the original algorithm used by Mac to show images in early era

In [ ]:
@numba.jit("f4[:,:,:](f4[:,:,:])", nopython=True, nogil=True)
def atkinson(image):
    frac = 8
    Lx, Ly, Lc = image.shape
    for j in range(Ly):
        for i in range(Lx):
            for c in range(Lc):
                rounded = round(image[i,j,c])
                err = image[i,j,c] - rounded
                image[i,j,c] = rounded
                if i<Lx-1: image[i+1,j,c] += err / frac
                if i<Lx-2: image[i+2,j,c] += err /frac
                if j<Ly-1:
                    image[i,j+1,c] += err / frac
                    if i>0: image[i-1,j+1,c] += err / frac
                    if i<Lx-1: image[i+1,j+1,c] += err / frac
                if j<Ly-2: image[i,j+2,c] += err / frac      
    return image